<a href="https://colab.research.google.com/github/codewithselva/industrial-copper-modelling/blob/main/Capstone_Industrial_Copper_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **About the Data:**
1. `id`: This column likely serves as a unique identifier for each transaction or item, which can be useful for tracking and record-keeping.
2. `item_date`: This column represents the date when each transaction or item was recorded or occurred. It's important for tracking the timing of business activities.
3. `quantity tons`: This column indicates the quantity of the item in tons, which is essential for inventory management and understanding the volume of products sold or produced.
4. `customer`: The "customer" column refers to the name or identifier of the customer who either purchased or ordered the items. It's crucial for maintaining customer relationships and tracking sales.
5. `country`: The "country" column specifies the country associated with each customer. This information can be useful for understanding the geographic distribution of customers and may have implications for logistics and international sales.
6. `status`: The "status" column likely describes the current status of the transaction or item. This information can be used to track the progress of orders or transactions, such as "Draft" or "Won."
7. `item type`: This column categorizes the type or category of the items being sold or produced. Understanding item types is essential for inventory categorization and business reporting.
8. `application`: The "application" column defines the specific use or application of the items. This information can help tailor marketing and product development efforts.
9. `thickness`: The "thickness" column provides details about the thickness of the items. It's critical when dealing with materials where thickness is a significant factor, such as metals or construction materials.
10. `width`: The "width" column specifies the width of the items. It's important for understanding the size and dimensions of the products.
11. `material_ref`: This column appears to be a reference or identifier for the material used in the items. It's essential for tracking the source or composition of the products.
12. `product_ref`: The "product_ref" column seems to be a reference or identifier for the specific product. This information is useful for identifying and cataloging products in a standardized way.
13. `delivery date`: This column records the expected or actual delivery date for each item or transaction. It's crucial for managing logistics and ensuring timely delivery to customers.
14. `selling_price`: The "selling_price" column represents the price at which the items are sold. This is a critical factor for revenue generation and profitability analysis.

**Approach: **
1. Data Understanding: Identify the types of variables (continuous, categorical) and their distributions. Some rubbish values are present in ‘Material_Reference’ which starts with ‘00000’ value which should be converted into null. Treat reference columns as categorical variables. INDEX may not be useful.
2. Data Preprocessing:
Handle missing values with mean/median/mode.
Treat Outliers using IQR or Isolation Forest from sklearn library.
Identify Skewness in the dataset and treat skewness with appropriate data transformations, such as log transformation(which is best suited to transform target variable-train, predict and then reverse transform it back to original scale eg:dollars), boxcox transformation, or other techniques, to handle high skewness in continuous variables.
Encode categorical variables using suitable techniques, such as one-hot encoding, label encoding, or ordinal encoding, based on their nature and relationship with the target variable.
3. EDA: Try visualizing outliers and skewness(before and after treating skewness) using Seaborn’s boxplot, distplot, violinplot.
4. Feature Engineering: Engineer new features if applicable, such as aggregating or transforming existing features to create more informative representations of the data. And drop highly correlated columns using SNS HEATMAP.
5. Model Building and Evaluation:
Split the dataset into training and testing/validation sets.
Train and evaluate different classification models, such as ExtraTreesClassifier, XGBClassifier, or Logistic Regression, using appropriate evaluation metrics such as accuracy, precision, recall, F1 score, and AUC curve.
Optimize model hyperparameters using techniques such as cross-validation and grid search to find the best-performing model.
Interpret the model results and assess its performance based on the defined problem statement.
Same steps for Regression modelling.(note: dataset contains more noise and linearity between independent variables so itll perform well only with tree based models)
6. Model GUI: Using streamlit module, create interactive page with
   (1) task input( Regression or Classification) and
   (2) create an input field where you can enter each column value except ‘Selling_Price’ for regression model and  except ‘Status’ for classification model.
   (3) perform the same feature engineering, scaling factors, log/any transformation steps which you used for training ml model and predict this new data from streamlit and display the output.
7. Tips: Use pickle module to dump and load models such as encoder(onehot/ label/ str.cat.codes /etc), scaling models(standard scaler), ML models. First fit and then transform in separate line and use transform only for unseen data
Eg: scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test_new) #unseen data


In [58]:
import gdown
import pandas as pd
import chardet
import numpy as np

In [5]:
# Google Drive file ID
file_id = "18eR6DBe5TMWU9FnIewaGtsepDbV4BOyr"

# URL of the file on Google Drive
url = f'https://drive.google.com/uc?id={file_id}'

# Destination file path to save the downloaded file
output_path = '/content/data.csv'

# Download the file
gdown.download(url, output_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=18eR6DBe5TMWU9FnIewaGtsepDbV4BOyr
To: /content/data.csv
100%|██████████| 17.2M/17.2M [00:00<00:00, 82.6MB/s]


'/content/data.csv'

In [74]:
excel_file_path = '/content/copper_data_set.xlsx'

# Specify the sheet name or index
sheet_name_or_index = 'Result 1'  # or use 0 for the first sheet

# Read the Excel file with a specific sheet into a Pandas DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name_or_index)


In [14]:
# Display the first few rows of the DataFrame

df.head()


,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,20210401.0,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,DEQ1 S460MC,1670798778,20210701.0,854.00
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,20210401.0,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,0000000000000000000000000000000000104991,1668701718,20210401.0,1047.00
2,E140FF1B-2407-4C02-A0DD-780A093B1158,20210401.0,386.127949,30153963.0,30.0,Won,WI,28.0,0.38,952.0,S0380700,628377,20210101.0,644.33
3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,20210401.0,202.411065,30349574.0,32.0,Won,S,59.0,2.30,1317.0,DX51D+ZM310MAO 2.3X1317,1668701718,20210101.0,768.00
4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,20210401.0,785.526262,30211560.0,28.0,Won,W,10.0,4.00,2000.0,2_S275JR+AR-CL1,640665,20210301.0,577.00


In [19]:
# Total number of records in the data set

len(df)

181673

In [75]:
# Display the info of the DataFrame

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181673 entries, 0 to 181672
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             181671 non-null  object 
 1   item_date      181672 non-null  float64
 2   quantity tons  181673 non-null  object 
 3   customer       181672 non-null  float64
 4   country        181645 non-null  float64
 5   status         181671 non-null  object 
 6   item type      181673 non-null  object 
 7   application    181649 non-null  float64
 8   thickness      181672 non-null  float64
 9   width          181673 non-null  float64
 10  material_ref   103754 non-null  object 
 11  product_ref    181673 non-null  int64  
 12  delivery date  181672 non-null  float64
 13  selling_price  181672 non-null  float64
dtypes: float64(8), int64(1), object(5)
memory usage: 19.4+ MB


**Inference:**
1. Total number of records: 181673
2. item_date field is in float64 Dtype which needs to be converted into date Dtype
3.


In [76]:
# Create a copy to avoid modifying the original DataFrame
cleaned_df = df.copy()

In [77]:
cleaned_df.head()

,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,20210401.0,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,DEQ1 S460MC,1670798778,20210701.0,854.00
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,20210401.0,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,0000000000000000000000000000000000104991,1668701718,20210401.0,1047.00
2,E140FF1B-2407-4C02-A0DD-780A093B1158,20210401.0,386.127949,30153963.0,30.0,Won,WI,28.0,0.38,952.0,S0380700,628377,20210101.0,644.33
3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,20210401.0,202.411065,30349574.0,32.0,Won,S,59.0,2.30,1317.0,DX51D+ZM310MAO 2.3X1317,1668701718,20210101.0,768.00
4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,20210401.0,785.526262,30211560.0,28.0,Won,W,10.0,4.00,2000.0,2_S275JR+AR-CL1,640665,20210301.0,577.00


In [78]:
# Update the 'item_date' for the specific 'id'
cleaned_df.loc[cleaned_df['id'] == '175B56C3-CDF1-4BD4-BC83-C1BF1FEAD8B8', 'item_date'] = np.nan

# Display the updated DataFrame
print(cleaned_df[cleaned_df['id'] == '175B56C3-CDF1-4BD4-BC83-C1BF1FEAD8B8'])

                                      id  item_date quantity tons    customer  \
52  175B56C3-CDF1-4BD4-BC83-C1BF1FEAD8B8        NaN     27.743221  30162161.0   

    country status item type  application  thickness   width   material_ref  \
52     77.0    Won         S          4.0        1.1  1300.0  DX51D+Z100 MA   

    product_ref  delivery date  selling_price  
52    164141591     20210601.0         1046.0  


In [79]:

cleaned_df.sort_values(by='item_date', ascending=True, inplace=True)
cleaned_df.head()

,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
105485,40203729-1A96-481E-9B71-3FF672C27F0B,20191919.0,101.742899,30210087.0,26.0,Lost,S,42.0,3.00,1494.0,NaN,1668701718,20210201.0,795.00
181672,AD0CA853-AE3C-4B2F-9FBB-8B0B965F84BC,20200702.0,406.686538,30200854.0,25.0,Won,W,41.0,0.71,1240.0,0000000000000000000000000000000001005439,164141591,20200701.0,607.00
180732,A55830C0-0ACF-44F6-8FB6-AB8F8D71213E,20200702.0,7.195474,30200854.0,25.0,Won,W,41.0,0.55,1220.0,0000000000000000000000000000000001005539,611993,20200701.0,1024.00
180733,EAFE5B2F-CB1A-42CC-BA84-72EF59B520EA,20200702.0,2.819479,30267349.0,84.0,Won,W,10.0,3.00,1226.0,1AMS420NTL34G,1670798778,20200701.0,600.38
180734,BED57CA3-4396-41B9-BEE2-A4A28C2473C6,20200702.0,51.488791,30200854.0,25.0,Won,W,41.0,0.45,1220.0,0000000000000000000000000000000001000092,611993,20200701.0,846.00


In [80]:
# Uniform date format
cleaned_df['item_date'] = pd.to_datetime(cleaned_df['item_date'], format='%Y%m%d')
cleaned_df['delivery date'] = pd.to_datetime(cleaned_df['delivery date'], format='%Y%m%d')

/usr/local/lib/python3.10/dist-packages/pandas/core/tools/datetimes.py:1256: RuntimeWarning: invalid value encountered in cast
  masked_result = calc(carg[mask].astype(np.float64).astype(np.int64))


ValueError: unconverted data remains: 19

In [81]:
cleaned_df.head()

,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
105485,40203729-1A96-481E-9B71-3FF672C27F0B,20191919.0,101.742899,30210087.0,26.0,Lost,S,42.0,3.00,1494.0,NaN,1668701718,20210201.0,795.00
181672,AD0CA853-AE3C-4B2F-9FBB-8B0B965F84BC,20200702.0,406.686538,30200854.0,25.0,Won,W,41.0,0.71,1240.0,0000000000000000000000000000000001005439,164141591,20200701.0,607.00
180732,A55830C0-0ACF-44F6-8FB6-AB8F8D71213E,20200702.0,7.195474,30200854.0,25.0,Won,W,41.0,0.55,1220.0,0000000000000000000000000000000001005539,611993,20200701.0,1024.00
180733,EAFE5B2F-CB1A-42CC-BA84-72EF59B520EA,20200702.0,2.819479,30267349.0,84.0,Won,W,10.0,3.00,1226.0,1AMS420NTL34G,1670798778,20200701.0,600.38
180734,BED57CA3-4396-41B9-BEE2-A4A28C2473C6,20200702.0,51.488791,30200854.0,25.0,Won,W,41.0,0.45,1220.0,0000000000000000000000000000000001000092,611993,20200701.0,846.00


In [22]:
# Handling missing values
cleaned_df.dropna(inplace=True)

In [26]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103750 entries, 0 to 181672
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             103750 non-null  object 
 1   item_date      103750 non-null  float64
 2   quantity tons  103750 non-null  object 
 3   customer       103750 non-null  float64
 4   country        103750 non-null  float64
 5   status         103750 non-null  object 
 6   item type      103750 non-null  object 
 7   application    103750 non-null  float64
 8   thickness      103750 non-null  float64
 9   width          103750 non-null  float64
 10  material_ref   103750 non-null  object 
 11  product_ref    103750 non-null  int64  
 12  delivery date  103750 non-null  float64
 13  selling_price  103750 non-null  float64
dtypes: float64(8), int64(1), object(5)
memory usage: 11.9+ MB


In [25]:
# Handling duplicate rows
cleaned_df.drop_duplicates(inplace=True)

In [27]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103750 entries, 0 to 181672
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             103750 non-null  object 
 1   item_date      103750 non-null  float64
 2   quantity tons  103750 non-null  object 
 3   customer       103750 non-null  float64
 4   country        103750 non-null  float64
 5   status         103750 non-null  object 
 6   item type      103750 non-null  object 
 7   application    103750 non-null  float64
 8   thickness      103750 non-null  float64
 9   width          103750 non-null  float64
 10  material_ref   103750 non-null  object 
 11  product_ref    103750 non-null  int64  
 12  delivery date  103750 non-null  float64
 13  selling_price  103750 non-null  float64
dtypes: float64(8), int64(1), object(5)
memory usage: 11.9+ MB


In [28]:
# Dealing with negative selling prices
cleaned_df = cleaned_df[cleaned_df['selling_price'] >= 0]

In [29]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103748 entries, 0 to 181672
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             103748 non-null  object 
 1   item_date      103748 non-null  float64
 2   quantity tons  103748 non-null  object 
 3   customer       103748 non-null  float64
 4   country        103748 non-null  float64
 5   status         103748 non-null  object 
 6   item type      103748 non-null  object 
 7   application    103748 non-null  float64
 8   thickness      103748 non-null  float64
 9   width          103748 non-null  float64
 10  material_ref   103748 non-null  object 
 11  product_ref    103748 non-null  int64  
 12  delivery date  103748 non-null  float64
 13  selling_price  103748 non-null  float64
dtypes: float64(8), int64(1), object(5)
memory usage: 11.9+ MB


In [31]:
# Handling outliers (consider replacing 3 with the appropriate threshold)
cleaned_df = cleaned_df[(cleaned_df['quantity tons'].between(cleaned_df['quantity tons'].quantile(0.01), cleaned_df['quantity tons'].quantile(0.99))) &
                        (cleaned_df['thickness'].between(cleaned_df['thickness'].quantile(0.01), cleaned_df['thickness'].quantile(0.99)))]

In [32]:
cleaned_df.head()

,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,20210401.0,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,DEQ1 S460MC,1670798778,20210701.0,854.00
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,20210401.0,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,0000000000000000000000000000000000104991,1668701718,20210401.0,1047.00
2,E140FF1B-2407-4C02-A0DD-780A093B1158,20210401.0,386.127949,30153963.0,30.0,Won,WI,28.0,0.38,952.0,S0380700,628377,20210101.0,644.33
3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,20210401.0,202.411065,30349574.0,32.0,Won,S,59.0,2.30,1317.0,DX51D+ZM310MAO 2.3X1317,1668701718,20210101.0,768.00
4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,20210401.0,785.526262,30211560.0,28.0,Won,W,10.0,4.00,2000.0,2_S275JR+AR-CL1,640665,20210301.0,577.00


In [33]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99700 entries, 0 to 181672
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             99700 non-null  object 
 1   item_date      99700 non-null  float64
 2   quantity tons  99700 non-null  object 
 3   customer       99700 non-null  float64
 4   country        99700 non-null  float64
 5   status         99700 non-null  object 
 6   item type      99700 non-null  object 
 7   application    99700 non-null  float64
 8   thickness      99700 non-null  float64
 9   width          99700 non-null  float64
 10  material_ref   99700 non-null  object 
 11  product_ref    99700 non-null  int64  
 12  delivery date  99700 non-null  float64
 13  selling_price  99700 non-null  float64
dtypes: float64(8), int64(1), object(5)
memory usage: 13.4+ MB


<ipython-input-35-96de89788f4a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['item_date'] = pd.to_datetime(cleaned_df['item_date'])
<ipython-input-35-96de89788f4a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['delivery date'] = pd.to_datetime(cleaned_df['delivery date'])


In [36]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99700 entries, 0 to 181672
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             99700 non-null  object        
 1   item_date      99700 non-null  datetime64[ns]
 2   quantity tons  99700 non-null  object        
 3   customer       99700 non-null  float64       
 4   country        99700 non-null  float64       
 5   status         99700 non-null  object        
 6   item type      99700 non-null  object        
 7   application    99700 non-null  float64       
 8   thickness      99700 non-null  float64       
 9   width          99700 non-null  float64       
 10  material_ref   99700 non-null  object        
 11  product_ref    99700 non-null  int64         
 12  delivery date  99700 non-null  datetime64[ns]
 13  selling_price  99700 non-null  float64       
dtypes: datetime64[ns](2), float64(6), int64(1), object(5)
memory usage: 1

In [37]:
cleaned_df.head()

,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,1970-01-01 00:00:00.020210401,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,DEQ1 S460MC,1670798778,1970-01-01 00:00:00.020210701,854.00
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,1970-01-01 00:00:00.020210401,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,0000000000000000000000000000000000104991,1668701718,1970-01-01 00:00:00.020210401,1047.00
2,E140FF1B-2407-4C02-A0DD-780A093B1158,1970-01-01 00:00:00.020210401,386.127949,30153963.0,30.0,Won,WI,28.0,0.38,952.0,S0380700,628377,1970-01-01 00:00:00.020210101,644.33
3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,1970-01-01 00:00:00.020210401,202.411065,30349574.0,32.0,Won,S,59.0,2.30,1317.0,DX51D+ZM310MAO 2.3X1317,1668701718,1970-01-01 00:00:00.020210101,768.00
4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,1970-01-01 00:00:00.020210401,785.526262,30211560.0,28.0,Won,W,10.0,4.00,2000.0,2_S275JR+AR-CL1,640665,1970-01-01 00:00:00.020210301,577.00


In [39]:
# Checking for consistency in categorization
# Assuming 'status', 'item_type', 'application' are categorical columns
cleaned_df['status'] = cleaned_df['status'].astype('category')
cleaned_df['item type'] = cleaned_df['item type'].astype('category')
cleaned_df['application'] = cleaned_df['application'].astype('category')

<ipython-input-39-a64258355fe1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['status'] = cleaned_df['status'].astype('category')
<ipython-input-39-a64258355fe1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['item type'] = cleaned_df['item type'].astype('category')
<ipython-input-39-a64258355fe1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [40]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99700 entries, 0 to 181672
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             99700 non-null  object        
 1   item_date      99700 non-null  datetime64[ns]
 2   quantity tons  99700 non-null  object        
 3   customer       99700 non-null  float64       
 4   country        99700 non-null  float64       
 5   status         99700 non-null  category      
 6   item type      99700 non-null  category      
 7   application    99700 non-null  category      
 8   thickness      99700 non-null  float64       
 9   width          99700 non-null  float64       
 10  material_ref   99700 non-null  object        
 11  product_ref    99700 non-null  int64         
 12  delivery date  99700 non-null  datetime64[ns]
 13  selling_price  99700 non-null  float64       
dtypes: category(3), datetime64[ns](2), float64(5), int64(1), object(3)
me

In [41]:
cleaned_df.head()

,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,1970-01-01 00:00:00.020210401,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,DEQ1 S460MC,1670798778,1970-01-01 00:00:00.020210701,854.00
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,1970-01-01 00:00:00.020210401,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,0000000000000000000000000000000000104991,1668701718,1970-01-01 00:00:00.020210401,1047.00
2,E140FF1B-2407-4C02-A0DD-780A093B1158,1970-01-01 00:00:00.020210401,386.127949,30153963.0,30.0,Won,WI,28.0,0.38,952.0,S0380700,628377,1970-01-01 00:00:00.020210101,644.33
3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,1970-01-01 00:00:00.020210401,202.411065,30349574.0,32.0,Won,S,59.0,2.30,1317.0,DX51D+ZM310MAO 2.3X1317,1668701718,1970-01-01 00:00:00.020210101,768.00
4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,1970-01-01 00:00:00.020210401,785.526262,30211560.0,28.0,Won,W,10.0,4.00,2000.0,2_S275JR+AR-CL1,640665,1970-01-01 00:00:00.020210301,577.00


In [42]:
# Some rubbish values are present in ‘Material_Reference’ which starts with ‘00000’ value which should be converted into null
cleaned_df['material_ref'] = cleaned_df['material_ref'].apply(lambda x: None if str(x).startswith('0000') else x)


<ipython-input-42-a716da89d761>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['material_ref'] = cleaned_df['material_ref'].apply(lambda x: None if str(x).startswith('0000') else x)


In [43]:
cleaned_df.head()

,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,1970-01-01 00:00:00.020210401,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,DEQ1 S460MC,1670798778,1970-01-01 00:00:00.020210701,854.00
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,1970-01-01 00:00:00.020210401,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,None,1668701718,1970-01-01 00:00:00.020210401,1047.00
2,E140FF1B-2407-4C02-A0DD-780A093B1158,1970-01-01 00:00:00.020210401,386.127949,30153963.0,30.0,Won,WI,28.0,0.38,952.0,S0380700,628377,1970-01-01 00:00:00.020210101,644.33
3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,1970-01-01 00:00:00.020210401,202.411065,30349574.0,32.0,Won,S,59.0,2.30,1317.0,DX51D+ZM310MAO 2.3X1317,1668701718,1970-01-01 00:00:00.020210101,768.00
4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,1970-01-01 00:00:00.020210401,785.526262,30211560.0,28.0,Won,W,10.0,4.00,2000.0,2_S275JR+AR-CL1,640665,1970-01-01 00:00:00.020210301,577.00


In [44]:
# material_ref - is a category
cleaned_df['material_ref'] = cleaned_df['material_ref'].astype('category')

<ipython-input-44-586c429caf61>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['material_ref'] = cleaned_df['material_ref'].astype('category')


In [45]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99700 entries, 0 to 181672
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             99700 non-null  object        
 1   item_date      99700 non-null  datetime64[ns]
 2   quantity tons  99700 non-null  object        
 3   customer       99700 non-null  float64       
 4   country        99700 non-null  float64       
 5   status         99700 non-null  category      
 6   item type      99700 non-null  category      
 7   application    99700 non-null  category      
 8   thickness      99700 non-null  float64       
 9   width          99700 non-null  float64       
 10  material_ref   78024 non-null  category      
 11  product_ref    99700 non-null  int64         
 12  delivery date  99700 non-null  datetime64[ns]
 13  selling_price  99700 non-null  float64       
dtypes: category(4), datetime64[ns](2), float64(5), int64(1), object(2)
me